<a href="https://colab.research.google.com/github/ananyakaligal/My-Machine-Learning-Journey/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
import time

In [ ]:
# Define constants
batchSize = 256
numEpochs = 15

In [ ]:
def load_mnist(train_samples=None, test_samples=None):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    # Normalize the pixel values to be between 0 and 1
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Reshape to add the channel dimension
    x_train = x_train.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)

    # Convert labels to one-hot encoding
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

    if train_samples:
        # Randomly sample the training data
        indices_train = np.random.choice(len(x_train), train_samples, replace=False)
        x_train = x_train[indices_train]
        y_train = y_train[indices_train]

    if test_samples:
        # Randomly sample the testing data
        indices_test = np.random.choice(len(x_test), test_samples, replace=False)
        x_test = x_test[indices_test]
        y_test = y_test[indices_test]

    return x_train, y_train, x_test, y_test

# Example usage: Load 1000 training samples and 200 test samples
x_train, y_train, x_test, y_test = load_mnist(train_samples=10000, test_samples=2000)

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# Print the shapes of the datasets
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

x_train shape: (10000, 28, 28, 1)
y_train shape: (10000, 10)
x_test shape: (2000, 28, 28, 1)
y_test shape: (2000, 10)


In [ ]:
import numpy as np

def format_data(data):
    indices = []
    values = []
    batch_size = data.shape[0]
    height = data.shape[1]
    width = data.shape[2]
    channels = data.shape[3] if len(data.shape) > 3 else 1  # Add channel dimension if present

    for i in range(batch_size):
        for row in range(height):
            for col in range(width):
                for channel in range(channels):
                    indices.append([i + 1, row + 1, col + 1, channel + 1])  # Adjust indices to start from 1
                    values.append(float(data[i, row, col, channel]))

    return np.array(indices), np.array(values)


In [ ]:
# Formatting data
formatted_x_train_indices, formatted_x_train_values = format_data(x_train)
formatted_x_test_indices, formatted_x_test_values = format_data(x_test)

In [ ]:
# Define a function to print samples of the formatted data
def print_sample(indices, values, sample_size=30):
    for i in range(sample_size):
        print(f"Indices: {indices[i]}, Value: {values[i]}")

# Print samples
print("Sample of train_indices and train_values:")
print_sample(formatted_x_train_indices, formatted_x_train_values)

Sample of train_indices and train_values:
Indices: [1 1 1 1], Value: 0.0
Indices: [1 1 2 1], Value: 0.0
Indices: [1 1 3 1], Value: 0.0
Indices: [1 1 4 1], Value: 0.0
Indices: [1 1 5 1], Value: 0.0
Indices: [1 1 6 1], Value: 0.0
Indices: [1 1 7 1], Value: 0.0
Indices: [1 1 8 1], Value: 0.0
Indices: [1 1 9 1], Value: 0.0
Indices: [ 1  1 10  1], Value: 0.0
Indices: [ 1  1 11  1], Value: 0.0
Indices: [ 1  1 12  1], Value: 0.0
Indices: [ 1  1 13  1], Value: 0.0
Indices: [ 1  1 14  1], Value: 0.0
Indices: [ 1  1 15  1], Value: 0.0
Indices: [ 1  1 16  1], Value: 0.0
Indices: [ 1  1 17  1], Value: 0.0
Indices: [ 1  1 18  1], Value: 0.0
Indices: [ 1  1 19  1], Value: 0.0
Indices: [ 1  1 20  1], Value: 0.0
Indices: [ 1  1 21  1], Value: 0.0
Indices: [ 1  1 22  1], Value: 0.0
Indices: [ 1  1 23  1], Value: 0.0
Indices: [ 1  1 24  1], Value: 0.0
Indices: [ 1  1 25  1], Value: 0.0
Indices: [ 1  1 26  1], Value: 0.0
Indices: [ 1  1 27  1], Value: 0.0
Indices: [ 1  1 28  1], Value: 0.0
Indices: [1 2 

In [ ]:
def create_tensor_from_data(indices, values, shape):
    # Convert indices and values to tensors
    indices_tensor = tf.constant(indices, dtype=tf.int64)
    values_tensor = tf.constant(values, dtype=tf.float32)

    # Create sparse tensor
    sparse_tensor = tf.sparse.SparseTensor(
        indices=indices_tensor,
        values=values_tensor,
        dense_shape=shape
    )

    # Convert sparse tensor to dense tensor
    dense_tensor = tf.sparse.to_dense(sparse_tensor)
    return dense_tensor